In [1]:
# Next-day volume forecast
from data_utils import set_seed, read_and_merge, build_features_nextday, split_by_date, fit_transform_scalers, make_sequences_generic, time_aware_train_val_split
from model_utils import build_lstm, train
from eval_utils import regression_report_log_and_orig, plot_history, scatter_actual_vs_pred, plot_timeseries

WINDOW = 30; BATCH_SIZE = 64; EPOCHS = 100; LR = 1e-3; SEED = 42
set_seed(SEED)

# Load from dedicated csvs used in your notebook
df, split_date = read_and_merge("train_volume_vix.csv", "test_volume_vix.csv")
df = build_features_nextday(df)



2025-11-13 10:22:48.695598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df.head()

,date,sh_volume,vix_close,target_volume,log_volume_t,vix_lag1,vix_change,vix_5d_ma,y_log
0,2004-01-08,5.293304e+09,15.61,4.770808e+09,22.389708,15.50,0.11,16.710,22.285781
1,2004-01-09,4.770808e+09,16.75,4.286810e+09,22.285781,15.61,1.14,16.416,22.178809
2,2004-01-12,4.286810e+09,16.82,4.557797e+09,22.178809,16.75,0.07,16.282,22.240105
3,2004-01-13,4.557797e+09,18.04,4.202376e+09,22.240105,16.82,1.22,16.544,22.158916
4,2004-01-14,4.202376e+09,16.75,4.606945e+09,22.158916,18.04,-1.29,16.794,22.250831


In [ ]:


train_df, test_df = split_by_date(df, split_date)
feature_cols = ["log_volume_t", "vix_close", "vix_lag1", "vix_change", "vix_5d_ma"]
train_s, test_s, scaler = fit_transform_scalers(train_df, test_df, feature_cols)

X_train, y_train, _ = make_sequences_generic(train_s, WINDOW, feature_cols)
X_test,  y_test,  idx_test = make_sequences_generic(test_s,  WINDOW, feature_cols)
X_tr, y_tr, X_val, y_val = time_aware_train_val_split(X_train, y_train, val_frac=0.15)

model = build_lstm(WINDOW, n_features=len(feature_cols), lr=LR)
hist = train(model, X_tr, y_tr, X_val, y_val, epochs=EPOCHS, batch_size=BATCH_SIZE)

y_pred_log = model.predict(X_test).ravel()
report = regression_report_log_and_orig(y_test, y_pred_log)
print(report)

plot_history(hist, title="Training History — Next-day")
scatter_actual_vs_pred(y_test, y_pred_log, title="Next-day — Actual vs Pred (log)")
plot_timeseries(idx_test, np.exp(y_test), np.exp(y_pred_log), ylabel="Volume", title="Next-day — Original scale")